# Caffe学习系列：solver及其配置

solver算是caffe的核心的核心，它协调着整个模型的运作。caffe程序运行必带的一个参数就是solver配置文件。运行代码一般为：
```
caffe train --solver=*_solver.prototxt
```

在Deep Learning中，往往loss function是非凸的，没有解析解，我们需要通过优化方法来求解。solver的主要作用就是交替调用前向（forward）算法和后向（backward）算法来更新参数，从而最小化loss，实际上就是一种迭代的优化算法。

到目前的版本，caffe提供了六种优化算法来求解最优参数，在solver配置文件中，通过设置type类型来选择。
- Stochastic Gradient Descent (type: "SGD")
- AdaDelta (type: "AdaDelta")
- Adaptive Gradient (type: "AdaGrad")
- Adam (type: "Adam")
- Nesterov's Accelerated Gradient (type: "Nesterov")
- RMSProp (type: "RMSProp")

具体的每种方法的介绍，请看本系列的下一篇文章，本文着重介绍solver配置文件的编写。

solver的流程：
1. 设计好需要优化的对象，以及用于学习的训练网络和用于评估的测试网络。（通过调用另外一个配置文件prototxt来进行）
2. 通过forward和backward迭代的进行优化来更新参数。
3. 定期的评价测试网络。（可设定多少次训练后，进行一次测试）
4. 在优化过程中显示模型和solver的状态。

在每一次迭代过程中，solver做了这几步工作：
1. 调用forward算法来计算最终的输出值，以及对应的loss
2. 调用backward算法来计算每层的梯度
3. 根据选用的solver方法，利用梯度进行参数更新
4. 记录并保存每次迭代的学习率、快照，以及对应的状态

接下来，我们先看一个示例：
```
net: "examples/mnist/lenet_train_test.prototxt"
test_iter: 100
test_interval: 500
base_lr: 0.01
momentum: 0.9
type: SGD
weight_decay: 0.0005
lr_policy: "inv"
gamma: 0.0001
power: 0.75
display: 100
max_iter: 20000
snapshot: 5000
snapshot_prefix: "examples/mnist/lenet"
solver_mode: CPU
```

接下来，我们对每一行进行详细解释：
```
net: "examples/mnist/lenet_train_test.prototxt"
```
设置深度网络模型。每一个模型就是一个net，需要在一个专门的配置文件中对net进行配置，每个net由许多的layer所组成。每一个layer的具体配置方式可参考前面的系列文章中的第一篇到第四篇。注意的是：文件的路径要从caffe的根目录开始，其他的所有配置都是这样。

也可用train_net和test_net来对训练模型和测试模型分别设定。例如：
```
train_net: "examples/hdf5_classification/logreg_auto_train.prototxt"
test_net: "examples/hdf5_classification/logreg_auto_test.prototxt"
```

接下来第二行：
```
test_iter: 100
```
这个要与test layer中的batch_size结合起来理解。mnist数据中测试样本总数为10000，一次性执行全部数据效率很低，因此我们将测试数据分成几个批次来执行，每个批次的数量就是batch_size。假设我们设置batch_size为100，则需要迭代100次才能将10000个数据全部执行完。因此test_iter设置为100。执行完一次全部数据，称之为一个epoch。

```
test_interval: 500
```
测试间隔。也就是每训练500次，才进行一次测试。

```
base_lr: 0.01
lr_policy: "inv"
gamma: 0.0001
power: 0.75
```
这四行可以放在一起理解，用于学习率的设置。只要是梯度下降法来求解优化，都会有一个学习率，也叫步长。base_lr用于设置基础学习率，在迭代的过程中，可以对学习率进行调整。怎么样进行调整，就是调整的策略，由lr_policy来设置。

lr_policy可以设置为下面这些值，相应的学习率计算为：
- fixed：保持base_lr不变。
- step：如果设置为step，则还需要设置一个stepsize，返回$base\_lr \times gamma^{floor(\cfrac{iter}{stepsize})}$，其中iter表示当前的迭代次数。
- exp：返回$base\_lr \times gamma^{iter}$，iter为当前迭代次数。
- inv：如果设置为inv，还需要设置一个power，返回$base\_lr \times (1 + gamma \times iter)^{-power}$。
- multistep：如果设置为multistep，则还需要设置一个stepvalue。这个参数和step很相似，step是均匀等间隔变化，而multistep则是根据stepvalue值变化。
- poly：学习率进行多项式误差，返回$base\_lr \times (1 - \cfrac{iter}{max\_iter})^{power}$。
- sigmoid：学习率进行sigmoid衰减，返回$base\_lr \times (\cfrac{1}{1 + e^{-gamma \times (iter - stepsize)}})$

multistep示例：
```
base_lr: 0.01
momentum: 0.9
weight_decay: 0.0005
# The learning rate policy
lr_policy: "multistep"
gamma: 0.9
stepvalue: 5000
stepvalue: 7000
stepvalue: 8000
stepvalue: 9000
stepvalue: 9500
```

接下来的参数：
```
momentum: 0.9
```
上一次梯度更新的权重，具体可参看下一篇文章。

```
type: SGD
```
优化算法选择。这一行可以省略掉，因为默认值就是SGD。总共有六种方法可选择，在本文的开头已介绍。

```
weight_decay: 0.0005
```
权重衰减项，防止过拟合的一个参数。

```
display: 100
```
每训练100次，在屏幕上显示一次。如果设置为0，则不显示。

```
max_iter: 20000
```
最大迭代次数。这个数设置太小，会导致没有收敛，精确度很低。设置太大，会导致震荡，浪费时间。

```
snapshot: 5000
snapshot_prefix: "examples/mnist/lenet"
```
快照。将训练出来的model和solver状态进行保存，snapshot用于设置训练多少次后进行保存，默认为0，不保存。snapshot_prefix设置保存路径。还可以设置snapshot_diff，是否保存梯度值，默认为false，不保存。也可以设置snapshot_format，保存的类型。有两种选择：HDF5和BINARYPROTO，默认为BINARYPROTO。

```
solver_mode: CPU
```
设置运行模式。默认为GPU，如果你没有GPU，则需要改成CPU，否则会出错。

注意：以上的所有参数都是可选参数，都有默认值。根据solver方法(type)的不同，还有一些其他的参数，在此不一一列举了。